# VGG16 vs. ResNet50

In [1]:
import keras
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense

In [2]:
!wget  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2025-06-06 16:16:19--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  30.9MB/s    in 8.1s    

2025-06-06 16:16:28 (30.9 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [3]:
!unzip -qo concrete_data_week4.zip

In [4]:
data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_gen = data_gen.flow_from_directory('/content/concrete_data_week4/train', batch_size=100, target_size =(224,224),
                                         class_mode='categorical')
valid_gen = data_gen.flow_from_directory('/content/concrete_data_week4/valid', batch_size=100, target_size =(224,224),
                                         class_mode='categorical')

model = Sequential()
model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

for layer in model.layers[0].layers:
    layer.trainable = False

model.fit(train_gen, steps_per_epoch=len(train_gen), validation_steps=len(valid_gen), epochs=2, verbose=1)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 176s 494ms/step - accuracy: 0.9268 - loss: 0.1766
Epoch 2/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 177s 498ms/step - accuracy: 0.9956 - loss: 0.0178


In [5]:
from google.colab import files
uploaded = files.upload()
from tensorflow.keras.models import load_model
res_model = load_model(list(uploaded.keys())[0])

Saving classifier_resnet_model.keras to classifier_resnet_model.keras


In [6]:
image_gen = ImageDataGenerator()
test_gen = image_gen.flow_from_directory('/content/concrete_data_week4/test', target_size=(224,224), shuffle=False)

Found 500 images belonging to 2 classes.


In [7]:
loss, acc_res = res_model.evaluate(test_gen, batch_size=100)
loss_, acc_vgg = model.evaluate(test_gen, batch_size=100)

16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 168ms/step - accuracy: 0.9908 - loss: 0.0392
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 848ms/step - accuracy: 0.9797 - loss: 0.0614


In [8]:
print(f"ResNet50 Performance:\nLoss: {loss:.4f}\nAccuracy: {acc_res}")
print(f"VGG16 Performance:\nLoss: {loss_:.4f}\nAccuracy: {acc_vgg}")

ResNet50 Performance:
Loss: 0.0758
Accuracy: 0.9779999852180481
VGG16 Performance:
Loss: 0.1453
Accuracy: 0.9480000138282776


In [9]:
import numpy as np
z_res = res_model.predict(test_gen)
predicted_classes = np.argmax(z_res, axis=1)

class_mapping = {0: "Positive", 1: "Negative"}
for i in range(5):
    print(class_mapping[predicted_classes[i]])

16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 248ms/step
Positive
Positive
Positive
Positive
Positive
